# Aprendizaje Profundo para Procesamiento deSeñales de Imagen y Vídeo (APPIV)
## Master in Data Science / Máster en Ciencia de Datos
## Universidad Autonoma de Madrid

## **P2: multiple object tracking for video sequences (template for assignment)**

This task for the assignment "práctica 2" is to **goal is to analyze and improve the tracking performance of the baseline multi-object tracker provided in the tutorial**. Your submission consists on a report in PDF format and this notebook (plus any additional source code files needed). You can *cut&paste as needed from the tutorial notebook* but please identify clearly your contributions with respect to the tutorial.

Your submission should adhere to the following restrictions (see the tutorial for further info on each aspect):
* The tracker must be based on the tracking-by-detection scheme, so an object detector must be applied. 
* For evaluation, you must use the [MOT16](https://motchallenge.net/data/MOT16/) dataset. In particular, you must use the train/test partition provided in the [material](http://www-vpu.eps.uam.es/~jcs/APPIV/appiv_p2_material.zip) based on the original `MOT16-train`(the only one that has ground-truth available). 
* The evaluation should be quantitative employing well-known metrics such as MOTA, MOTP, TP, FP, FN and IDswitches.


Author1: Íñigo Gómez (innigo.gomezc@estudiante.uam.es)

Author2: Jon Zorrilla (jon.zorrilla@estudiante.uam.es)


# 1 Setup
Add here all the required setup and libraries to run your tracker...

In [1]:
!pip install git+https://github.com/cheind/py-motmetrics.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/cheind/py-motmetrics.git to /tmp/pip-req-build-ncd_thdk
  Running command git clone --filter=blob:none --quiet https://github.com/cheind/py-motmetrics.git /tmp/pip-req-build-ncd_thdk
  Resolved https://github.com/cheind/py-motmetrics.git to commit beb864a56d6055047c4d6e015188fcc24aca05b7
  Preparing metadata (setup.py) ... done
  Created wheel for motmetrics: filename=motmetrics-1.4.0-py3-none-any.whl size=162181 sha256=6b696883b57d8d4e724da4c10e4d42e07b0799519cdb6fa7dafb5bcc273b705b
  Stored in directory: /tmp/pip-ephem-wheel-cache-rsbpsvt2/wheels/fa/43/80/267b90f028883ee680a5c3260e08a80a8ed9f0fc0afdf2e704
Successfully built motmetrics


In [2]:

%load_ext autoreload
%autoreload 2
%matplotlib inline

!pip3 install tqdm lap
!pip3 install torch==1.13.0+cu116 torchvision==0.14.0+cu116 torchaudio===0.13.0+cu116 --extra-index-url https://download.pytorch.org/whl/cu116 #Pytorch and requirements
!pip3 install https://github.com/timmeinhardt/py-motmetrics/archive/fix_pandas_deprecating_warnings.zip #Evaluation metrics for MOT
!pip install git+https://github.com/cheind/py-motmetrics.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 39.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lap: filename=lap-0.4.0-cp310-cp310-linux_x86_64.whl size=1655212 sha256=971644fa1ce71ad814e66e0ee0b012be832b9deec6bdba33bc3f08f1f9c12273
  Stored in directory: /root/.cache/pip/wheels/00/42/2e/9dfe19270eea279d79e84767ff0d7b8082c3bf776cad00e83d
Successfully built lap
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 870.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 38.8 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu11

In [3]:
import torch, torchvision, torchaudio
print(torch.__version__)
print(torchvision.__version__)
print(torchaudio.__version__)

1.13.0+cu116
0.14.0+cu116
0.13.0+cu116


In [4]:
# Connet to Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


Here we provide you the link for the new material (https://drive.google.com/uc?export=download&id=1_G6Opx0TA6KSB12Jv-FX-ovYn7pQmK2V), you should change the content of the variable *material_dir* to fit the path of where the zip file is located in your GDrive



In [5]:
import os, subprocess

# path to download in Google Drive
material_dir = '/content/gdrive/MyDrive/MASTER/P2_video/appiv_p2_material.zip'
download_dir = '/content/gdrive/My Drive/downloads/' 

# file with the material for the assignment
datafile = 'appiv_p2_material.zip' #with extension

In [6]:
import os, subprocess, zipfile
from tqdm.autonotebook import tqdm

# path to working directory (local, will be deleted when VM disconnected)
working_dir = '/content/work/'                      

# copy downloaded file to the working directory
print('Copying file from \'{}\' to \'{}\'...'.format(datafile,material_dir,working_dir))
if os.path.isfile(os.path.join(working_dir,datafile)) == False: 
  !rsync -ah --progress "$material_dir" "$working_dir"
else:
  print('already exists!')

# uncompress/unzip file in the working directory
print('\nUnzipping file \'{}\' in directory \'{}\'...'.format(datafile, working_dir), end='')
if os.path.isdir(os.path.join(working_dir,'data')) == False: 
  with zipfile.ZipFile(os.path.join(working_dir,datafile)) as zf:
    for member in tqdm(zf.infolist(), desc='Extracting '):
        try:
          zf.extract(member, working_dir)
        except zipfile.error as e:
            pass
else:
  print('already unzipped!')



Copying file from 'appiv_p2_material.zip' to '/content/gdrive/MyDrive/MASTER/P2_video/appiv_p2_material.zip'...
sending incremental file list
created directory /content/work
appiv_p2_material.zip
          1.08G 100%   41.25MB/s    0:00:24 (xfr#1, to-chk=0/1)

Unzipping file 'appiv_p2_material.zip' in directory '/content/work/'...

Extracting :   0%|          | 0/8232 [00:00<?, ?it/s]

In [7]:
import sys,os

# add the path to the system so we can import the tracker
sys.path.append(os.path.join(working_dir,'appiv_p2_material/src/'))

# test that we can successfully import the tracker
from tracker.tracker import Tracker
print('If \'from tracker.tracker import Tracker\' shows no errors, you can import and execute the sample tracker code.\n')
print('Source files of the tracker are available at the directory \'{}\':'.format(working_dir+'src/tracker/'))
!ls $working_dir/src/tracker

print('object_detector.py --> defines the object detector to be employed for tracking (FRCNN_FPN)')
print('tracker.py         --> defines the architecture for developing trackers (i.e. common functions)')
print('data_obj_detect.py --> utility for handing datasets/sequences for object detection')
print('data_track.py      --> utility for handing datasets/sequences for object tracking')
print('utils.py           --> Utilities for evaluation and plotting')

If 'from tracker.tracker import Tracker' shows no errors, you can import and execute the sample tracker code.

Source files of the tracker are available at the directory '/content/work/src/tracker/':
ls: cannot access '/content/work//src/tracker': No such file or directory
object_detector.py --> defines the object detector to be employed for tracking (FRCNN_FPN)
tracker.py         --> defines the architecture for developing trackers (i.e. common functions)
data_obj_detect.py --> utility for handing datasets/sequences for object detection
data_track.py      --> utility for handing datasets/sequences for object tracking
utils.py           --> Utilities for evaluation and plotting


# 2 Dataset

Add here all the required code to load the dataset...


In [8]:
#list the contents of the 'train' directory
train_dir = os.path.join(working_dir,'appiv_p2_material/data/MOT16/train')
print('Train directory:')
!ls $train_dir

#list the contents of the 'test' directory
test_dir = os.path.join(working_dir,'appiv_p2_material/data/MOT16/test')
print('Test directory:')
!ls $test_dir

Train directory:
MOT16-02  MOT16-04  MOT16-05  MOT16-09
Test directory:
MOT16-10  MOT16-11  MOT16-13


In [9]:
import matplotlib.pyplot as plt
from tracker.data_track import MOT16Sequences

seq_name = 'MOT16-02'
seq_name = 'MOT16-train'

data_dir = os.path.join(working_dir,'appiv_p2_material/data/MOT16')
sequences = MOT16Sequences(seq_name, data_dir, load_seg=True)

#3 Object detector
Add here all the required code to load the detector employed by the tracker...


In [10]:
# path for the source code of the tracker
model_dir=working_dir+'appiv_p2_material/models/'

print('Model files of the tracker are available at the directory \'{}\':'.format(model_dir))
!ls "$model_dir"

Model files of the tracker are available at the directory '/content/work/appiv_p2_material/models/':
faster_rcnn_fpn.model


In [11]:
import torch

#select GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

#location of the model file
obj_detect_model_file = os.path.join(working_dir, "appiv_p2_material/models/faster_rcnn_fpn.model")

#threshold for non maximum suppression
obj_detect_nms_thresh = 0.5

#detector has been trained for two classes
num_classes=2 # 1 class (person) + background (see https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html)

In [12]:
from tracker.object_detector import FRCNN_FPN

# object detector
obj_detect = FRCNN_FPN(num_classes=num_classes, nms_thresh=obj_detect_nms_thresh)
obj_detect_state_dict = torch.load(obj_detect_model_file,map_location=lambda storage, loc: storage)
obj_detect.load_state_dict(obj_detect_state_dict)

# prints the architecture and sets the model to evaluation mode.
obj_detect.eval()

# loads detector to CPU or GPU (if available)
obj_detect.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'backbone_name' and 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


FRCNN_FPN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu)

In [13]:
from torch.utils.data import DataLoader
from tracker.data_obj_detect import MOT16ObjDetect
from tracker.utils import (evaluate_obj_detect, obj_detect_transforms)

#load train set for the MOT16 data (images and ground-truth bounding boxes)
dataset_test = MOT16ObjDetect(os.path.join(working_dir, 'appiv_p2_material/data/MOT16/train'), obj_detect_transforms(train=False))
def collate_fn(batch):
    return tuple(zip(*batch))
data_loader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, num_workers=2,collate_fn=collate_fn)

#evaluation
evaluate_obj_detect(obj_detect, data_loader_test)

  0%|          | 0/3012 [00:00<?, ?it/s]

AP: 0.8629069023981812 Prec: 0.9091963295351416 Rec: 0.938094990376112 TP: 54099.0 FP: 5403.0


#4 Multi-object tracking
Add here all the required code to load and run your tracker...

In [15]:
import torch
from tracker.object_detector import FRCNN_FPN

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# object detector
obj_detect = FRCNN_FPN(num_classes=num_classes, nms_thresh=obj_detect_nms_thresh)
obj_detect_state_dict = torch.load(obj_detect_model_file,map_location=lambda storage, loc: storage)
obj_detect.load_state_dict(obj_detect_state_dict)
obj_detect.eval()     # set to evaluation mode
obj_detect.to(device) # load detector to GPU or CPU

# select dataset
data_dir = os.path.join(working_dir,'appiv_p2_material/data/MOT16')
seq_name_test = 'MOT16-train'
sequences_test = MOT16Sequences(seq_name_test, data_dir)
print('Loaded {:d} sequences for {:s}'.format(len(sequences_test),seq_name_test))

#output directory
output_dir = os.path.join(working_dir, 'output')

Loaded 4 sequences for MOT16-train


In [16]:
import torch
import numpy as np 

seed = 12345 #seed to allow repeatable results
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

In [17]:
import numpy as np
import motmetrics as mm
mm.lap.default_solver = 'lap'
from tracker.tracker import Tracker
        
# create a new tracker using the 'TrackerIoUAssignment' class
tracker = Tracker(obj_detect)
print('Tracker created!')

Tracker created!


In [18]:
import time
from tqdm.autonotebook import tqdm
from tracker.utils import get_mot_accum

time_total = 0
mot_accums = []
results_seq = {}

for seq in sequences_test:
    print(f"Tracking: {seq}")
    now = time.time()

    # restart tracker state for each sequence
    tracker.reset()
    
    #load data
    data_loader = DataLoader(seq, batch_size=1, shuffle=False)

    #run tracker
    for frame in tqdm(data_loader):
        tracker.step(frame)

    #keep results
    results = tracker.get_results()
    results_seq[str(seq)] = results

    #perform evaluation
    if seq.no_gt:
        print(f"No GT evaluation data available.")
    else:
        mot_accums.append(get_mot_accum(results, seq)) #compute and store eval metrics 

    time_total += time.time() - now

    print(f"Tracks found: {len(results)}")
    print(f"Runtime for {seq}: {time.time() - now:.1f} s.")

    #save results to output directory
    seq.write_results(results, os.path.join(output_dir))

Tracking: MOT16-02


  0%|          | 0/600 [00:00<?, ?it/s]

Tracks found: 42
Runtime for MOT16-02: 165.1 s.
Writing predictions to: /content/work/output/MOT16-02.txt
Tracking: MOT16-09


  0%|          | 0/525 [00:00<?, ?it/s]

Tracks found: 36
Runtime for MOT16-09: 147.1 s.
Writing predictions to: /content/work/output/MOT16-09.txt
Tracking: MOT16-04


  0%|          | 0/1050 [00:00<?, ?it/s]

Tracks found: 119
Runtime for MOT16-04: 290.8 s.
Writing predictions to: /content/work/output/MOT16-04.txt
Tracking: MOT16-05


  0%|          | 0/837 [00:00<?, ?it/s]

Tracks found: 169
Runtime for MOT16-05: 128.1 s.
Writing predictions to: /content/work/output/MOT16-05.txt


In [19]:
from tracker.utils import evaluate_mot_accums

print(f"Runtime for all test sequences: {time_total:.1f} s.")
if mot_accums:
    evaluate_mot_accums(mot_accums,
                        [str(s) for s in sequences_test if not s.no_gt],
                        generate_overall=True)

Runtime for all test sequences: 731.2 s.
          IDF1   IDP   IDR  Rcll  Prcn  GT  MT  PT ML   FP    FN  IDs   FM  MOTA  MOTP  IDt IDa IDm
MOT16-02 21.9% 31.9% 16.7% 47.6% 90.8%  62  10  33 19  899  9744  609  214 39.4% 0.092  612  17  31
MOT16-09 23.5% 25.5% 21.9% 70.0% 81.5%  26  14  11  1  845  1596  258  143 49.3% 0.115  244  23  15
MOT16-04 21.4% 22.6% 20.4% 78.1% 86.8%  83  42  29 12 5654 10426 1906  363 62.2% 0.108 1862  52  29
MOT16-05 41.0% 47.3% 36.1% 65.5% 85.7% 133  38  77 18  753  2387  322  188 49.9% 0.150  288  87  62
OVERALL  23.4% 26.4% 21.0% 69.2% 86.9% 304 104 150 50 8151 24153 3095  908 54.8% 0.109 3006 179 137


In [20]:
import matplotlib.pyplot as plt
from tracker.utils import plot_sequence

plot_sequence(results_seq['MOT16-13'],
              [s for s in sequences_test if str(s) == 'MOT16-13'][0],
              first_n_frames=3)

KeyError: ignored

In [21]:
#copy output files to download directory in personal Google Drive
!cp -r $output_dir "$download_dir"

#show output files
print('Output tracking results:')
path = os.path.join(download_dir,"output")
!ls "$path"


Output tracking results:
MOT16-02.txt  MOT16-05.txt  MOT16-10.txt  MOT16-13.txt
MOT16-04.txt  MOT16-09.txt  MOT16-11.txt
